# PPTXファイルの情報を取得するサンプル

## 事前準備


- `/contents/00_setup` のノートブックを実行して、 PowerPoint のファイルを Volume に配置してください。

## 利用するライブラリをインストール

In [0]:
%pip install python-pptx lxml -q

In [0]:
%pip install "markitdown[pdf, docx, pptx]" -q

In [0]:
dbutils.library.restartPython()

## 共通設定の読み取りとライブラリのインポート

In [0]:
%run ../contents/00_config

## ファイルの確認

In [0]:
# Volume に書かれたことを確認
print("-- medalion_site.metadata.json")
pptx_path = source_dir + "/landing/raw_files"
pptx_path += "/audit__ingest_timestamp=2025-10-14T10:07:33Z"
pptx_path += "/drive/root:/test/files/databricks_overview_by_ai.pptx"
display(dbutils.fs.ls(pptx_path))

## MarkItDown にて LLM を使わないずに情報を抽出

In [0]:
from markitdown import MarkItDown

md = MarkItDown()
result = md.convert(pptx_path)

In [0]:
print(result.text_content[:1200])

## MarkItDown にて LLM を使って情報を抽出

In [0]:
import os

from openai import OpenAI

token = (
    dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
)
ws_url = "https://"
ws_url += spark.conf.get("spark.databricks.workspaceUrl")

client = OpenAI(
    api_key=token,
    base_url=f"{ws_url}/serving-endpoints",
)

In [0]:
# Databricks Free Edition で実施した場合には結果の出力の 6 分程度分程度かかる
md = MarkItDown(
    llm_client=client,
    llm_model="databricks-gpt-oss-20b",
    system_prompt="概要を教えて",
)
result = md.convert(pptx_path)

In [0]:
print(result.text_content[:1200])

## スライドごとに分割をする

In [0]:
import re
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, IntegerType, StringType


# スライド分割関数
def split_slides(content):
    slides = re.split(r"<!--\s*Slide number:\s*(\d+)\s*-->", content)
    result = []
    for i in range(1, len(slides), 2):
        slide_number = int(slides[i])
        slide_content = slides[i + 1].strip()
        result.append((slide_number, slide_content))
    return result

In [0]:
from markitdown import MarkItDown

md = MarkItDown()
result = md.convert(pptx_path)
slides_data = split_slides(result.text_content[:1200])

In [0]:
import pprint

print(slides_data)

In [0]:
# Spark DataFrameに変換
schema = StructType(
    [
        StructField("slide", IntegerType(), True),
        StructField("content", StringType(), True),
    ]
)
df = spark.createDataFrame(slides_data, schema=schema)
df.createOrReplaceTempView("slides")
df.display()

## AI_QUERYで要約を生成する

In [0]:
from markitdown import MarkItDown

md = MarkItDown()
result = md.convert(pptx_path)
subset_resulst = result.text_content[:2000]

df = spark.createDataFrame([result.text_content[:2000]], "content string")
df.createOrReplaceTempView("slides")
df.display()

In [0]:
summary_prompt = "次のテキストを日本語のマークダウン形式で要約してください。\n"
df = spark.sql(
    f"""
WITH summary AS (
    SELECT 
        ai_query(
            "databricks-gpt-oss-20b",
            CONCAT('{summary_prompt}', content)
        )  AS content_summary
    FROM slides
)
SELECT * FROM summary
"""
)

In [0]:
print(df.collect()[0].content_summary)

## python-pptx ライブラリにより PPTX のメタ情報を抽出する

In [0]:
from pptx import Presentation

prs = Presentation(pptx_path)
prop = prs.core_properties
print("title: ", prop.title)
print("category: ", prop.category)
print("created: ", prop.created)
print("modified: ", prop.modified)

In [0]:
# end